In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [2]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        demoPaths.append(os.path.join(dir_path, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    mapData, killData, nadeData, flashData, roundData, damageData = [], [], [], [], [], []
    for x in parsers:
        x.parse()
        mapData.append(x.parse(return_type="df"))
        killData.append(x._parse_kills(return_type="df"))
        nadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (mapData, killData, nadeData, flashData, roundData, damageData)

In [5]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [26]:
def createSidedCSVDF(players, damageData, killData, roundData):
    locationNames = []
    playerNames = []

    Kills = []
    Deaths = []
    KdDifferential = []
    DamageDealt = []
    DamageTaken = []
    DamageDifferential = []
    sides = []
    totalRounds = []
    dataframe = {}
    for x in players.keys():
        for y in killData[killData['AttackerName'].isin(players[x]) | killData['VictimName'].isin(players[x])].VictimAreaName.value_counts().index:
            
            tempCTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'CT'])
            tempCTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'CT'])
            tempCTKdDifferential = tempCTKills - tempCTDeaths
            
            tempCTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'CT'].HpDamageTaken.sum()
            tempCTDamageDifferential = tempCTDamageDealt - tempCTDamageTaken
                                                                                                             
            tempTKills = len(killData[killData['AttackerName'].isin(players[x])][killData['AttackerAreaName'] == y][killData['AttackerSide'] == 'T'])
            tempTDeaths = len(killData[killData['VictimName'].isin(players[x])][killData['VictimAreaName'] == y][killData['VictimSide'] == 'T'])
            tempTKdDifferential = tempTKills - tempTDeaths
                                                                                                             
            tempTDamageDealt = damageData[damageData['AttackerName'].isin(players[x])][damageData['AttackerAreaName'] == y][damageData['AttackerSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageTaken = damageData[damageData['VictimName'].isin(players[x])][damageData['VictimAreaName'] == y][damageData['VictimSide'] == 'T'].HpDamageTaken.sum()
            tempTDamageDifferential = tempTDamageDealt - tempTDamageTaken
            
            tempTotalRounds = 0
                                                    
            for z in killData[killData.AttackerName.isin(players[x])].MatchId.value_counts().index:
                tempTotalRounds += roundData[roundData['MatchId'] == z].iloc[-1].RoundNum
            
            locationNames.append(y)
            locationNames.append(y)
            
            playerNames.append(x)
            playerNames.append(x)
            
            Kills.append(tempCTKills)
            Kills.append(tempTKills)
            
            
            Deaths.append(tempCTDeaths)
            Deaths.append(tempTDeaths)
            
            
            KdDifferential.append(tempCTKdDifferential)
            KdDifferential.append(tempTKdDifferential)
            
            
            DamageDealt.append(tempCTDamageDealt)
            DamageTaken.append(tempCTDamageTaken)
            DamageDifferential.append(tempCTDamageDifferential)
            
            DamageDealt.append(tempTDamageDealt)
            DamageTaken.append(tempTDamageTaken)
            DamageDifferential.append(tempTDamageDifferential)
            
            sides.append('CT')
            sides.append('T')
            
            
            totalRounds.append(tempTotalRounds)
            totalRounds.append(tempTotalRounds)
    
    dataframe['Location'] = locationNames
    dataframe['Player'] = playerNames
    dataframe['Kills'] = Kills
    dataframe['Deaths'] = Deaths
    dataframe['K/D'] = KdDifferential
    dataframe['Damage Dealt'] = DamageDealt
    dataframe['Damage Taken'] = DamageTaken
    dataframe['Damage Differential'] = DamageDifferential
    dataframe['Side'] = sides
    dataframe['Total Rounds'] = totalRounds
    
    return pandas.DataFrame.from_dict(dataframe, orient='columns')
                        
                

In [6]:
dir_path = "./demos/Dreamhack"

In [51]:
dreamhack_output_path = './damageDifferentials/Dreamhack/dreamhackMirageDamageDifferentials.csv'
kills_output_path = './damageDifferentials/Dreamhack/killsDreamhack.csv'
damages_output_path = './damageDifferentials/Dreamhack/damagesDreamhack.csv'
rounds_output_path = './damageDifferentials/Dreamhack/roundsDreamhack.csv'

In [27]:
mirageMappingPath = './mappings/mappings/'

In [28]:
mappingFile = os.path.join(mirageMappingPath, os.listdir(mirageMappingPath)[0])

In [29]:
file = open(mappingFile, 'r')
lines = file.readlines()
file.close()

In [30]:
for x in range(len(lines)):
    if(lines[x][-1] == '\n'):
        lines[x] = lines[x][:-1]

In [31]:
mappedPlaceNames = {}
for x in lines:
    if(x != ''):
            mappedPlaceNames[x.split(':')[0]] = x.split(':')[1]

In [7]:
demoPaths = getDemoPaths(dir_path)

In [9]:
len(demoPaths)

42

In [10]:
demoParsers = generateDemoParsers(demoPaths)

17:57:42 [INFO] Go version>=1.14.0
17:57:42 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/astralis-vs-complexity-m2-inferno.dem
17:57:42 [INFO] Setting demo id to astralis-vs-complexity-m2-inferno
17:57:42 [INFO] Setting parse rate to 128
17:57:42 [INFO] Go version>=1.14.0
17:57:42 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/astralis-vs-extra-salt-nuke.dem
17:57:42 [INFO] Setting demo id to astralis-vs-extra-salt-nuke
17:57:42 [INFO] Setting parse rate to 128
17:57:42 [INFO] Go version>=1.14.0
17:57:42 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/astralis-vs-heroic-m2-inferno.dem
17:57:42 [INFO] Setting demo id to astralis-vs-heroic-m2-inferno
17:57:42 [INFO] Setting parse rate to 128
17:57:42 [INFO] Go version>=1.14.0
17:57:42 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/extra-salt-vs-vitality-m3-nuke.de

17:57:44 [INFO] Go version>=1.14.0
17:57:44 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/natus-vincere-vs-mousesports-overpass.dem
17:57:44 [INFO] Setting demo id to natus-vincere-vs-mousesports-overpass
17:57:44 [INFO] Setting parse rate to 128
17:57:44 [INFO] Go version>=1.14.0
17:57:44 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/natus-vincere-vs-virtus-pro-m1-dust2.dem
17:57:44 [INFO] Setting demo id to natus-vincere-vs-virtus-pro-m1-dust2
17:57:44 [INFO] Setting parse rate to 128
17:57:44 [INFO] Go version>=1.14.0
17:57:44 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/Dreamhack/natus-vincere-vs-virtus-pro-m2-inferno.dem
17:57:44 [INFO] Setting demo id to natus-vincere-vs-virtus-pro-m2-inferno
17:57:44 [INFO] Setting parse rate to 128
17:57:44 [INFO] Go version>=1.14.0
17:57:44 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos

# Demo Parsers: 42


In [11]:
maps, kills, nades, flashs, rounds, damages = generateData(demoParsers)

17:57:46 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:57:46 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/astralis-vs-complexity-m2-inferno.dem
17:58:19 [INFO] Wrote demo parse output to astralis-vs-complexity-m2-inferno.json
17:58:19 [INFO] Reading in JSON from astralis-vs-complexity-m2-inferno.json
17:58:20 [INFO] JSON data loaded, available in the `json` attribute to parser
17:58:20 [INFO] Successfully parsed JSON output
17:58:20 [INFO] Successfully returned JSON output
17:58:20 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:58:20 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/astralis-vs-complexity-m2-inferno.dem
17:58:48 [INFO] Wrote demo parse output to astralis-vs-complexity-m2-inferno.json
17:58:48 [INFO] Reading in JSON from astralis-vs-complexity-m2-inferno.json
17:58:49 [INFO] JSON data loaded, available in the `json` attribute to parser
17:58:49 [INFO] Successfully p

18:01:07 [INFO] Parsed grenades to Pandas DataFrame
18:01:07 [INFO] Parsed flashes to Pandas DataFrame
18:01:07 [INFO] Parsed bomb_events to Pandas DataFrame
18:01:07 [INFO] Parsed frames to Pandas DataFrame
18:01:07 [INFO] Parsed player frames to Pandas DataFrame
18:01:07 [INFO] Returned dataframe output
18:01:07 [INFO] Parsed kills to Pandas DataFrame
18:01:07 [INFO] Parsed grenades to Pandas DataFrame
18:01:07 [INFO] Parsed flashes to Pandas DataFrame
18:01:07 [INFO] Parsed rounds to Pandas DataFrame
18:01:07 [INFO] Parsed damages to Pandas DataFrame
18:01:07 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:01:07 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/fiend-vs-spirit-m2-overpass.dem
18:01:46 [INFO] Wrote demo parse output to fiend-vs-spirit-m2-overpass.json
18:01:46 [INFO] Reading in JSON from fiend-vs-spirit-m2-overpass.json
18:01:47 [INFO] JSON data loaded, available in the `json` attribute to parser
18:01:47 [INFO] Succes

18:05:09 [INFO] Wrote demo parse output to furia-vs-natus-vincere-m3-mirage.json
18:05:09 [INFO] Reading in JSON from furia-vs-natus-vincere-m3-mirage.json
18:05:10 [INFO] JSON data loaded, available in the `json` attribute to parser
18:05:10 [INFO] Successfully parsed JSON output
18:05:10 [INFO] Successfully returned JSON output
18:05:10 [INFO] Parsed rounds to Pandas DataFrame
18:05:10 [INFO] Parsed kills to Pandas DataFrame
18:05:10 [INFO] Parsed damages to Pandas DataFrame
18:05:10 [INFO] Parsed grenades to Pandas DataFrame
18:05:10 [INFO] Parsed flashes to Pandas DataFrame
18:05:10 [INFO] Parsed bomb_events to Pandas DataFrame
18:05:10 [INFO] Parsed frames to Pandas DataFrame
18:05:10 [INFO] Parsed player frames to Pandas DataFrame
18:05:10 [INFO] Returned dataframe output
18:05:10 [INFO] Parsed kills to Pandas DataFrame
18:05:10 [INFO] Parsed grenades to Pandas DataFrame
18:05:10 [INFO] Parsed flashes to Pandas DataFrame
18:05:10 [INFO] Parsed rounds to Pandas DataFrame
18:05:10 

18:08:02 [INFO] JSON data loaded, available in the `json` attribute to parser
18:08:02 [INFO] Successfully parsed JSON output
18:08:02 [INFO] Successfully returned JSON output
18:08:02 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:08:02 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/g2-vs-mousesports-m1-vertigo_1.dem
18:08:19 [INFO] Wrote demo parse output to g2-vs-mousesports-m1-vertigo.json
18:08:19 [INFO] Reading in JSON from g2-vs-mousesports-m1-vertigo.json
18:08:19 [INFO] JSON data loaded, available in the `json` attribute to parser
18:08:19 [INFO] Successfully parsed JSON output
18:08:19 [INFO] Successfully returned JSON output
18:08:19 [INFO] Parsed rounds to Pandas DataFrame
18:08:19 [INFO] Parsed kills to Pandas DataFrame
18:08:19 [INFO] Parsed damages to Pandas DataFrame
18:08:19 [INFO] Parsed grenades to Pandas DataFrame
18:08:19 [INFO] Parsed flashes to Pandas DataFrame
18:08:19 [INFO] Parsed bomb_events to Pandas DataF

18:10:23 [INFO] Parsed damages to Pandas DataFrame
18:10:24 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:10:24 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/gambit-vs-g2-m3-dust2.dem
18:10:35 [INFO] Wrote demo parse output to gambit-vs-g2-m3-dust2.json
18:10:35 [INFO] Reading in JSON from gambit-vs-g2-m3-dust2.json
18:10:35 [INFO] JSON data loaded, available in the `json` attribute to parser
18:10:35 [INFO] Successfully parsed JSON output
18:10:35 [INFO] Successfully returned JSON output
18:10:35 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:10:35 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/gambit-vs-g2-m3-dust2.dem
18:10:46 [INFO] Wrote demo parse output to gambit-vs-g2-m3-dust2.json
18:10:46 [INFO] Reading in JSON from gambit-vs-g2-m3-dust2.json
18:10:47 [INFO] JSON data loaded, available in the `json` attribute to parser
18:10:47 [INFO] Successfully parsed JSON output
18:

18:13:45 [INFO] Parsed kills to Pandas DataFrame
18:13:45 [INFO] Parsed damages to Pandas DataFrame
18:13:45 [INFO] Parsed grenades to Pandas DataFrame
18:13:45 [INFO] Parsed flashes to Pandas DataFrame
18:13:45 [INFO] Parsed bomb_events to Pandas DataFrame
18:13:45 [INFO] Parsed frames to Pandas DataFrame
18:13:45 [INFO] Parsed player frames to Pandas DataFrame
18:13:45 [INFO] Returned dataframe output
18:13:45 [INFO] Parsed kills to Pandas DataFrame
18:13:45 [INFO] Parsed grenades to Pandas DataFrame
18:13:45 [INFO] Parsed flashes to Pandas DataFrame
18:13:45 [INFO] Parsed rounds to Pandas DataFrame
18:13:45 [INFO] Parsed damages to Pandas DataFrame
18:13:45 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:13:45 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/heroic-vs-natus-vincere-m2-nuke_1.dem
18:14:07 [INFO] Wrote demo parse output to heroic-vs-natus-vincere-m2-nuke.json
18:14:07 [INFO] Reading in JSON from heroic-vs-natus-vincere

18:16:47 [INFO] Successfully returned JSON output
18:16:47 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:16:47 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/mousesports-vs-faze-m2-nuke.dem
18:17:05 [INFO] Wrote demo parse output to mousesports-vs-faze-m2-nuke.json
18:17:05 [INFO] Reading in JSON from mousesports-vs-faze-m2-nuke.json
18:17:05 [INFO] JSON data loaded, available in the `json` attribute to parser
18:17:05 [INFO] Successfully parsed JSON output
18:17:05 [INFO] Successfully returned JSON output
18:17:05 [INFO] Parsed rounds to Pandas DataFrame
18:17:05 [INFO] Parsed kills to Pandas DataFrame
18:17:05 [INFO] Parsed damages to Pandas DataFrame
18:17:05 [INFO] Parsed grenades to Pandas DataFrame
18:17:05 [INFO] Parsed flashes to Pandas DataFrame
18:17:05 [INFO] Parsed bomb_events to Pandas DataFrame
18:17:05 [INFO] Parsed frames to Pandas DataFrame
18:17:05 [INFO] Parsed player frames to Pandas DataFrame
18:17:05 [INFO] Ret

18:20:56 [INFO] Parsed kills to Pandas DataFrame
18:20:56 [INFO] Parsed grenades to Pandas DataFrame
18:20:56 [INFO] Parsed flashes to Pandas DataFrame
18:20:56 [INFO] Parsed rounds to Pandas DataFrame
18:20:56 [INFO] Parsed damages to Pandas DataFrame
18:20:57 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:20:57 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/pain-vs-heroic-nuke.dem
18:21:15 [INFO] Wrote demo parse output to pain-vs-heroic-nuke.json
18:21:15 [INFO] Reading in JSON from pain-vs-heroic-nuke.json
18:21:15 [INFO] JSON data loaded, available in the `json` attribute to parser
18:21:15 [INFO] Successfully parsed JSON output
18:21:15 [INFO] Successfully returned JSON output
18:21:15 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:21:15 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/pain-vs-heroic-nuke.dem
18:21:34 [INFO] Wrote demo parse output to pain-vs-heroic-nuke.json
1

18:24:26 [INFO] Parsed kills to Pandas DataFrame
18:24:26 [INFO] Parsed damages to Pandas DataFrame
18:24:26 [INFO] Parsed grenades to Pandas DataFrame
18:24:26 [INFO] Parsed flashes to Pandas DataFrame
18:24:26 [INFO] Parsed bomb_events to Pandas DataFrame
18:24:26 [INFO] Parsed frames to Pandas DataFrame
18:24:26 [INFO] Parsed player frames to Pandas DataFrame
18:24:26 [INFO] Returned dataframe output
18:24:26 [INFO] Parsed kills to Pandas DataFrame
18:24:26 [INFO] Parsed grenades to Pandas DataFrame
18:24:26 [INFO] Parsed flashes to Pandas DataFrame
18:24:26 [INFO] Parsed rounds to Pandas DataFrame
18:24:26 [INFO] Parsed damages to Pandas DataFrame
18:24:26 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
18:24:26 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/Dreamhack/vitality-vs-big-m1-mirage.dem
18:24:46 [INFO] Wrote demo parse output to vitality-vs-big-m1-mirage.json
18:24:46 [INFO] Reading in JSON from vitality-vs-big-m1-mirage.json
18:24:

In [12]:
failedDemos = []
gambitMaps, gambitKills, gambitNades, gambitFlashs, gambitRounds, gambitDamages = [], [], [], [], [], []
for x in range(len(kills)):
    if(kills[x].shape[0] > 100):
        gambitMaps.append(maps[x])
        gambitKills.append(kills[x])
        gambitNades.append(nades[x])
        gambitFlashs.append(flashs[x])
        gambitRounds.append(rounds[x])
        gambitDamages.append(damages[x])
    else:
        failedDemos.append(x)

In [17]:
mirageMaps, mirageKills, mirageNades, mirageFlashs, mirageRounds, mirageDamages = [], [], [], [], [], []
for x in range(len(kills)):
    if(kills[x].MapName.iloc[0] == 'de_mirage'):
        mirageMaps.append(maps[x])
        mirageKills.append(kills[x])
        mirageNades.append(nades[x])
        mirageFlashs.append(flashs[x])
        mirageRounds.append(rounds[x])
        mirageDamages.append(damages[x])

In [13]:
killData = concat_data(gambitKills)
nadeData = concat_data(gambitNades)
flashData = concat_data(gambitFlashs)
roundData = concat_data(gambitRounds)
damageData = concat_data(gambitDamages)

In [35]:
mirageKillData = concat_data(mirageKills)
mirageNadeData = concat_data(mirageNades)
mirageFlashData = concat_data(mirageFlashs)
mirageRoundData = concat_data(mirageRounds)
mirageDamageData = concat_data(mirageDamages)

In [42]:
player_names = list(mirageKillData.AttackerName.value_counts().index)
player_name = {}
for x in player_names:
    player_name[x] = [x]

In [36]:
for x in range(len(mirageKillData)):
    if  mirageKillData.iloc[x]['AttackerAreaName'] != None:
        print
        mirageKillData.at[x, 'VictimAreaName'] = mappedPlaceNames[mirageKillData.iloc[x]['VictimAreaName']]
        mirageKillData.at[x, 'AttackerAreaName'] = mappedPlaceNames[mirageKillData.iloc[x]['AttackerAreaName']]

for x in range(len(mirageDamageData)):
    if  mirageDamageData.iloc[x]['AttackerAreaName'] != None:
        mirageDamageData.at[x, 'VictimAreaName'] = mappedPlaceNames[mirageDamageData.iloc[x]['VictimAreaName']]
        mirageDamageData.at[x, 'AttackerAreaName'] = mappedPlaceNames[mirageDamageData.iloc[x]['AttackerAreaName']]

In [43]:
mirageDifferentials = createSidedCSVDF(player_name, mirageDamageData, mirageKillData, mirageRoundData)

/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/nk095x/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Serie

In [44]:
mirageDifferentials

,Location,Player,Kills,Deaths,K/D,Damage Dealt,Damage Taken,Damage Differential,Side,Total Rounds
0,Default A,s1mple,0,0,0,0,25,-25,CT,78
1,Default A,s1mple,1,1,0,73,12,61,T,78
2,A Site,s1mple,0,1,-1,15,19,-4,CT,78
3,A Site,s1mple,1,3,-2,108,289,-181,T,78
4,Mid Close Connector,s1mple,0,1,-1,54,100,-46,CT,78
...,...,...,...,...,...,...,...,...,...,...
3229,Deep Jungle,acoR,0,0,0,0,0,0,T,23
3230,Tetris,acoR,0,0,0,0,0,0,CT,23
3231,Tetris,acoR,0,0,0,0,0,0,T,23
3232,Ticketbooth,acoR,0,1,-1,72,100,-28,CT,23


In [48]:
mirageDifferentials.to_csv(dreamhack_output_path)

In [52]:
mirageKillData.to_csv(kills_output_path)

In [53]:
mirageDamageData.to_csv(damages_output_path)

In [54]:
mirageRoundData.to_csv(rounds_output_path)